# 채권종목 데이터 DB저장

In [2]:
# 기본적인 라이브러리 import
import xlwings as xw
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from tqdm import tqdm
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
plt.rcParams["font.family"] = 'NanumGothic'

## 채권 종목정보 가져오기

In [4]:
# 채권 발행일과 만기일 기준으로 종목정보를 조회하는 bondlist 엑셀파일에서 이표채의 KR코드와 종목명을 가져옵니다.
sheet_bondinfo = xw.Book(r'bondlist.xlsx').sheets[0]
startcell = 'A8'
maxrow = sheet_bondinfo.range(startcell).end('down').row
maxcol = sheet_bondinfo.range(startcell).end('right').column
df = sheet_bondinfo.range(startcell,(maxrow,maxcol)).options(pd.DataFrame, index=False).value
df_coupon = df[df['이자종류'] == "이표채"]
codes = df_coupon['표준코드'].tolist()
names = [name[13:-1] for name in df_coupon['종목명'].tolist()]

## 채권데이터 조회 엑셀파일 만들기 - 처음 한번만 실행!!

In [ ]:
# 채권데이터엑셀 템플릿 시트를 여러개로 복제합니다. 처음에 한번만 하면 되는것임!!!!!!
sheets = xw.Book(r'bonddata.xlsx').sheets
for code, name in zip(codes, names):
    newsheet = sheets[0].copy(name=code) # 이표채 종목수만큼 시트를 복제, 시트명은 KR코드

In [ ]:
# 생성한 시트들을 loop돌면서 시트명(KR코드)을 받아와서 각시트 조회 종목코드셀('B3')에 넣어주기
# 이렇게 2단계로 진행해야 엑셀측에서 이상한 에러가 안난다.
sheets = xw.Book(r'db_bonddata.xlsx').sheets
for sheet in sheets:
    sheet.range('B3').value = sheet.name
# 전종목 데이터 조회 엑셀파일 완성

## 채권데이터 엑셀파일에서 데이터 불러오기

In [30]:
# 조회가 끝난 각 시트를 돌면서 데이터를 받아서 하나의 DataFrame으로 만든다.
sheets = xw.Book(r'bonddata.xlsx').sheets
keycell = 'A9'
downcell = 'B9'
df = pd.DataFrame()
for sheet in tqdm(sheets):
    maxrow = sheet.range(downcell).end('down').row
    maxcol = sheet.range(keycell).end('right').column
    bondname = sheet.range('B4').value[13:-1]
    tempdf = sheet.range(keycell,(maxrow,maxcol)).options(pd.DataFrame, index=False).value
    tempdf['종목명'] = bondname
    tempdf.set_index(['일자','종목명'], inplace=True)
    tempdf_notnull = tempdf[tempdf['민평3사 수익률(산출일) 당일'].notnull()]
    df = df.append(tempdf_notnull)
df


100%|██████████| 87/87 [00:37<00:00,  2.35it/s]


,,민평3사 수익률(산출일) 당일,민평3사 수익률(전일대비) 당일,민평3사 가격(산출일) 당일,민평3사 듀레이션(산출일) 당일,전체 순매수 거래량,외국인 순매수 거래량,은행 순매수 거래량,보험기금 순매수 거래량,자산운용(공모) 순매수 거래량,자산운용(사모) 순매수 거래량,...,연기금 대여,외국인 대여,기타 대여,증권 차입,보험 차입,투신 차입,은행 차입,연기금 차입,외국인 차입,기타 차입
일자,종목명,,,,,,,,,,,,,,,,,,,,,
2018-01-02,06-1,2.478,0.017,12591.02,6.6471,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
2018-01-03,06-1,2.489,0.011,12582.12,6.6433,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
2018-01-04,06-1,2.513,0.024,12563.22,6.6385,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
2018-01-05,06-1,2.496,-0.017,12578.27,6.6373,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
2018-01-08,06-1,2.519,0.023,12561.41,6.6270,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-19,21-5,1.993,NaN,9977.90,9.1725,-1.780000e+11,NaN,-2.270000e+11,4.000000e+10,1.000000e+09,NaN,...,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
2021-05-11,21-6,0.971,NaN,9972.97,2.0577,6.000000e+09,NaN,-8.000000e+09,1.300000e+10,1.000000e+09,NaN,...,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
2021-07-19,21-7,1.735,NaN,9982.21,4.9144,-1.200000e+10,-3.000000e+10,-2.000000e+11,2.971300e+10,1.882870e+11,NaN,...,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN


In [20]:
df.index.value_counts()

(2018-01-02 00:00:00, 06-1)    1
(2018-12-13 00:00:00, 17-7)    1
(2018-12-17 00:00:00, 17-7)    1
(2018-12-18 00:00:00, 17-7)    1
(2018-12-19 00:00:00, 17-7)    1
                              ..
(2019-06-05 00:00:00, 15-1)    1
(2019-06-07 00:00:00, 15-1)    1
(2019-06-10 00:00:00, 15-1)    1
(2019-06-11 00:00:00, 15-1)    1
(2021-09-24 00:00:00, 21-9)    1
Length: 54508, dtype: int64

In [28]:
df.loc(axis=0)[:,'15-4'].shape

(666, 33)

In [5]:
# DataFrame의 컬럼명을 적당한 이름들로 수정
df.columns = [
    '민평금리', '민평금리전일비', '민평가격', '듀레이션', 
    '전체순매수', '외국인순매수', '은행순매수', '보험기금순매수', '자산운용공모순매수', '자산운용사모순매수', '종금순매수', '정부순매수', '기타법인순매수', '개인순매수', 
    '장내거래량', '장외거래량',
    '대차거래', '대차상환', '대차잔량', 
    '증권대여', '보험대여', '투신대여', '은행대여', '연기금대여', '외국인대여', '기타대여', 
    '증권차입', '보험차입', '투신차입', '은행차입', '연기금차입', '외국인차입', '기타차입'
    ]

# 컬럼 추가 및 수정
# 대차증감 = 거래 - 상환
# 대차 전일잔량 삭제
df['대차증감'] = df['대차거래'] - df['대차상환']

In [6]:
# NA처리 (0으로 변경) - 문제소지는 민평금리 전일비가 일부(종목의 상장일) 0이 되는문제? ->딱히 문제는 없을듯 
df.fillna(0, inplace=True)
df.isna().sum()

민평금리         0
민평금리전일비      0
민평가격         0
듀레이션         0
전체순매수        0
외국인순매수       0
은행순매수        0
보험기금순매수      0
자산운용공모순매수    0
자산운용사모순매수    0
종금순매수        0
정부순매수        0
기타법인순매수      0
개인순매수        0
장내거래량        0
장외거래량        0
대차거래         0
대차상환         0
대차잔량         0
증권대여         0
보험대여         0
투신대여         0
은행대여         0
연기금대여        0
외국인대여        0
기타대여         0
증권차입         0
보험차입         0
투신차입         0
은행차입         0
연기금차입        0
외국인차입        0
기타차입         0
대차증감         0
dtype: int64

In [7]:
# 각 컬럼의 type을 적절하게 수정해준다.
df[['전체순매수', '외국인순매수', '은행순매수', '보험기금순매수', '자산운용공모순매수', '자산운용사모순매수', 
    '종금순매수', '정부순매수', '기타법인순매수', '개인순매수', '장내거래량', '장외거래량', 
    '대차거래', '대차상환', '대차잔량', 
    '증권대여', '보험대여', '투신대여', '은행대여', '연기금대여', '외국인대여', '기타대여', 
    '증권차입', '보험차입', '투신차입', '은행차입', '연기금차입', '외국인차입', '기타차입', '대차증감']] = df[['전체순매수', '외국인순매수', '은행순매수', '보험기금순매수', '자산운용공모순매수', '자산운용사모순매수', 
    '종금순매수', '정부순매수', '기타법인순매수', '개인순매수', '장내거래량', '장외거래량', 
    '대차거래', '대차상환', '대차잔량', 
    '증권대여', '보험대여', '투신대여', '은행대여', '연기금대여', '외국인대여', '기타대여', 
    '증권차입', '보험차입', '투신차입', '은행차입', '연기금차입', '외국인차입', '기타차입', '대차증감']].astype(np.int64)

In [8]:
df

,,민평금리,민평금리전일비,민평가격,듀레이션,전체순매수,외국인순매수,은행순매수,보험기금순매수,자산운용공모순매수,자산운용사모순매수,...,외국인대여,기타대여,증권차입,보험차입,투신차입,은행차입,연기금차입,외국인차입,기타차입,대차증감
일자,종목명,,,,,,,,,,,,,,,,,,,,,
2018-01-02,06-1,2.478,0.017,12591.02,6.6471,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2018-01-03,06-1,2.489,0.011,12582.12,6.6433,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2018-01-04,06-1,2.513,0.024,12563.22,6.6385,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2018-01-05,06-1,2.496,-0.017,12578.27,6.6373,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2018-01-08,06-1,2.519,0.023,12561.41,6.6270,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-15,21-9,2.060,-0.022,9700.73,16.5256,28000000000,0,22000000000,5000000000,1000000000,0,...,0,0,0,0,0,0,0,0,0,-3000
2021-09-16,21-9,2.068,0.008,9687.93,16.5195,0,0,1000000000,-6000000000,5000000000,0,...,0,0,0,0,0,0,0,0,0,0
2021-09-17,21-9,2.080,0.012,9669.83,16.5119,12000000000,0,1000000000,0,11000000000,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# 만든 DataFrame을 pickle 로 떨궈본다.
df.to_pickle('pkl_bonddata.pkl')

In [ ]:
# DataFrame을 sqlite 파일DB로 저장
import sqlite3
con = sqlite3.connect(r'bonddata.db')
df.to_sql('bonddata', con, if_exists='replace')

# 채권 상장잔액 증감 데이터 DB저장

## 채권 종목정보 가져오기 - 이전과 같음

## 채권 여러종목 상장잔액 조회 엑셀파일 만들기

In [ ]:
# 채권데이터엑셀 템플릿 시트를 여러개로 복제합니다.
sheets = xw.Book(r'db_bondbalance.xlsx').sheets
for code, name in tqdm(zip(codes, names)):
    newsheet = sheets[0].copy(name=code) # 이표채 종목수만큼 시트를 복제, 시트명은 KR코드

In [ ]:
# 생성한 시트들을 loop돌면서 시트명(KR코드)을 받아와서 각시트 조회 종목코드셀('B3')에 넣어주기
# 이렇게 2단계로 진행해야 엑셀측에서 이상한 에러가 안난다.
sheets = xw.Book(r'db_bondbalance.xlsx').sheets
for sheet in sheets:
    sheet.range('B3').value = sheet.name
# 전종목 데이터 조회 엑셀파일 완성

## 채권 상장잔액 엑셀파일에서 데이터 불러오기

In [ ]:
# 조회가 끝난 각 시트를 돌면서 데이터를 받아서 하나의 DataFrame으로 만든다.
sheets = xw.Book(r'db_bondbalance.xlsx').sheets
keycell = 'A9'
df = pd.DataFrame()
for sheet in tqdm(sheets):
    if sheet.range('A10').value != None: # 상장잔액증감이 없는 종목들 처리
        maxrow = sheet.range(keycell).end('down').row
        maxcol = sheet.range(keycell).end('right').column
        bondname = sheet.range('B4').value[13:-1]
        tempdf = sheet.range(keycell,(maxrow,maxcol)).options(pd.DataFrame, index=False).value
        tempdf['종목명'] = bondname
        tempdf.set_index(['일자','종목명'], inplace=True)
        # tempdf_notnull = tempdf[tempdf['민평3사 수익률(산출일) 당일'].notnull()]
        df = df.append(tempdf)
df


100%|██████████| 87/87 [00:03<00:00, 23.37it/s]


,,상장잔액증감
일자,종목명,
2018-01-24,06-1,-700000.0
2019-01-16,06-1,-400000.0
2019-05-22,06-1,-1000000.0
2018-01-24,07-3,-1200000.0
2019-01-16,07-3,-1000000.0
...,...,...
2021-07-20,21-7,2300000.0
2021-07-21,21-7,3000000.0
2021-07-22,21-7,6900000.0


In [ ]:
# 컬럼명은 문제가 없으므로 컬럼 타입만 int64로 변경
df[['상장잔액증감']] = df[['상장잔액증감']].astype(np.int64)
df

,,상장잔액증감
일자,종목명,
2018-01-24,06-1,-700000
2019-01-16,06-1,-400000
2019-05-22,06-1,-1000000
2018-01-24,07-3,-1200000
2019-01-16,07-3,-1000000
...,...,...
2021-07-20,21-7,2300000
2021-07-21,21-7,3000000
2021-07-22,21-7,6900000


In [ ]:
# 만든 DataFrame을 pickle 로 떨궈본다.
df.to_pickle('pkl_bondbalance.pkl')

In [ ]:
import sqlite3
con = sqlite3.connect(r'bondbalance.db')
df.to_sql('bondbalance', con, if_exists='replace')

# 시장금리 DB저장

In [ ]:
sheets = xw.Book(r'db_marketdata.xlsx').sheets
keycell = 'A4'
df = pd.DataFrame()
for sheet in tqdm(sheets):
    if sheet.range('A5').value != None: # 조회데이터 없는경우 처리
        maxrow = sheet.range(keycell).end('down').row
        maxcol = sheet.range(keycell).end('right').column
        tempdf = sheet.range(keycell,(maxrow,maxcol)).options(pd.DataFrame, index=False).value
        tempdf.set_index('일자', inplace=True)
        # tempdf_notnull = tempdf[tempdf['민평3사 수익률(산출일) 당일'].notnull()]
        df = df.append(tempdf)
df

100%|██████████| 1/1 [00:00<00:00,  7.30it/s]


,3년이하(당일),5년이하(당일),10년이하(당일),20년이하(당일),30년이하(당일),3년이하(전일대비),5년이하(전일대비),10년이하(전일대비),20년이하(전일대비),30년이하(전일대비)
일자,,,,,,,,,,
2011-01-03,3.500,4.150,4.570,4.733,0.000,0.114,0.070,0.060,0.053,0.000
2011-01-04,3.493,4.180,4.590,4.750,0.000,-0.007,0.030,0.020,0.017,0.000
2011-01-05,3.496,4.200,4.636,4.773,0.000,0.003,0.020,0.046,0.023,0.000
2011-01-06,3.580,4.290,4.710,4.823,0.000,0.084,0.090,0.074,0.050,0.000
2011-01-07,3.580,4.300,4.710,4.816,0.000,0.000,0.010,0.000,-0.007,0.000
...,...,...,...,...,...,...,...,...,...,...
2021-08-30,1.397,1.650,1.905,1.978,1.965,-0.008,-0.015,-0.032,-0.022,-0.027
2021-08-31,1.395,1.647,1.912,1.975,1.960,-0.002,-0.003,0.007,-0.003,-0.005
2021-09-01,1.410,1.672,1.942,1.995,1.975,0.015,0.025,0.030,0.020,0.015


In [ ]:
# DataFrame의 컬럼명을 적당한 이름들로 수정
df.columns = ['국고3년', '국고5년', '국고10년', '국고20년', '국고30년',
       '국고3년전일대비', '국고5년전일대비', '국고10년전일대비', '국고20년전일대비',
       '국고30년전일대비']

In [ ]:
# 만든 DataFrame을 pickle 로 떨궈본다.
df.to_pickle('pkl_marketdata.pkl')

In [ ]:
import sqlite3
con = sqlite3.connect(r'marketdata.db')
df.to_sql('marketdata', con, if_exists='replace')

# 입찰캘린더 DB저장

In [2]:
# 엑셀파일에서 DataFrame 만들기. 종목명 작업.
sheets = xw.Book(r'db_calendar.xlsx').sheets
keycell = 'A2'
sheet = sheets[0]
maxrow = sheet.range(keycell).end('down').row
maxcol = sheet.range(keycell).end('right').column
df = sheet.range(keycell,(maxrow,maxcol)).options(pd.DataFrame, index=False).value
df['종목명'] = df['전체종목명'].str[13:-1]

#발행년월 에서 발행년도, 발행월을 뽑아내는 다소 tricky한 부분
df['발행년도'] = df['발행년월'].astype(int)
df['발행월'] = np.round((df['발행년월']%1*100)).astype(int)
df.drop(['발행년월'], axis=1, inplace=True)
df.set_index(['일자','종목명'], inplace=True)
df.head()

,,표준코드,전체종목명,회차,만기,구분,발행예정액,낙찰금액,상장잔액,낙찰금리,응찰금액,응찰율,부분낙찰율,응찰금리하,응찰금리상,발행년도,발행월
일자,종목명,,,,,,,,,,,,,,,,
2021-09-28,21-9,KR103503GB99,국고01875-4109(21-9),신규,20.0,경쟁,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021,9
2021-09-27,21-7,KR103501GB91,국고01750-2609(21-7),2.0,5.0,경쟁,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021,9
2021-09-13,21-5,KR103502GB66,국고02000-3106(21-5),10.0,10.0,경쟁,2300000.0,2300000.0,15875000.0,2.02,6292000.0,273.6,7.8,1.98,2.045,2021,9
2021-09-09,20-10,KR103503GA90,국고01625-7009(20-10),8.0,50.0,경쟁,350000.0,350000.0,4560000.0,1.99,528000.0,150.9,52.5,1.50,2.015,2021,9
2021-09-07,21-8,KR103502GB90,국고01250-2309(21-8),2.0,2.0,경쟁,850000.0,850000.0,1367000.0,1.32,2354000.0,276.9,27.5,1.27,1.345,2021,9


In [3]:
# NA처리 (0으로 변경) - 공란인 것들이 좀 있음
df.fillna(0, inplace=True)

In [4]:

df[['만기', '발행예정액', '낙찰금액', '상장잔액', '응찰금액']] = df[['만기', '발행예정액', '낙찰금액', '상장잔액', '응찰금액']].astype(np.int64)
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 772 entries, (Timestamp('2021-09-28 00:00:00'), '21-9') to (Timestamp('2011-01-03 00:00:00'), '10-6')
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   표준코드    772 non-null    object 
 1   전체종목명   772 non-null    object 
 2   회차      772 non-null    object 
 3   만기      772 non-null    int64  
 4   구분      772 non-null    object 
 5   발행예정액   772 non-null    int64  
 6   낙찰금액    772 non-null    int64  
 7   상장잔액    772 non-null    int64  
 8   낙찰금리    772 non-null    float64
 9   응찰금액    772 non-null    int64  
 10  응찰율     772 non-null    float64
 11  부분낙찰율   772 non-null    float64
 12  응찰금리하   772 non-null    float64
 13  응찰금리상   772 non-null    float64
 14  발행년도    772 non-null    int32  
 15  발행월     772 non-null    int32  
dtypes: float64(5), int32(2), int64(5), object(4)
memory usage: 116.8+ KB


In [5]:
df

,,표준코드,전체종목명,회차,만기,구분,발행예정액,낙찰금액,상장잔액,낙찰금리,응찰금액,응찰율,부분낙찰율,응찰금리하,응찰금리상,발행년도,발행월
일자,종목명,,,,,,,,,,,,,,,,
2021-09-28,21-9,KR103503GB99,국고01875-4109(21-9),신규,20,경쟁,0,0,0,0.00,0,0.00,0.00,0.00,0.000,2021,9
2021-09-27,21-7,KR103501GB91,국고01750-2609(21-7),2.0,5,경쟁,0,0,0,0.00,0,0.00,0.00,0.00,0.000,2021,9
2021-09-13,21-5,KR103502GB66,국고02000-3106(21-5),10.0,10,경쟁,2300000,2300000,15875000,2.02,6292000,273.60,7.80,1.98,2.045,2021,9
2021-09-09,20-10,KR103503GA90,국고01625-7009(20-10),8.0,50,경쟁,350000,350000,4560000,1.99,528000,150.90,52.50,1.50,2.015,2021,9
2021-09-07,21-8,KR103502GB90,국고01250-2309(21-8),2.0,2,경쟁,850000,850000,1367000,1.32,2354000,276.90,27.50,1.27,1.345,2021,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011-02-07,10-6,KR10350170C2,국고03000-1312(10-6),3.0,3,경쟁,1500000,1500000,3400000,4.08,3437000,229.13,35.00,4.06,4.100,2011,2
2011-01-24,10-7,KR10350270C0,국고04750-3012(10-7),1.0,20,경쟁,970000,985000,1385000,4.90,2206000,227.42,0.00,4.88,4.950,2011,1
2011-01-17,10-3,KR1035027062,국고05000-2006(10-3),13.0,10,경쟁,1570000,1570000,11554000,4.69,4119000,262.36,19.20,4.68,4.730,2011,1


In [ ]:
# 만든 DataFrame을 pickle 로 떨궈본다.
df.to_pickle('pkl_calendar.pkl')

In [ ]:
import sqlite3
con = sqlite3.connect(r'calendar.db')
df.to_sql('calendar', con, if_exists='replace')

In [ ]:
df.info()

In [ ]:
df = pd.DataFrame()
for sht in tqdm(sheets_ktb):
    krcode = sht.range('F1').value
    bond_name = sht.range('G1').value
    maturity = sht.range('I1').value
    max_row = sht.range('B3').end('down').row
    temp_df = sht.range((3,1),(max_row,40)).options(pd.DataFrame, index=False).value
    temp_df['종목코드'] = krcode
    temp_df['종목명'] = bond_name[13:-1]
#     sht.name = bond_name[13:-1]
    temp_df['잔존만기'] = (maturity - temp_df['일자']).dt.days
    temp_df.set_index(['일자','종목명'], inplace=True)
    df = df.append(temp_df)

In [ ]:
# 조회가 끝난 각 시트를 돌면서 데이터를 받아서 하나의 DataFrame으로 만든다.
sheets = xw.Book(r'db_bonddata.xlsx').sheets
keycell = 'A9'
df = pd.DataFrame()
for sheet in tqdm(sheets):
    maxrow = sheet.range(keycell).end('down').row
    maxcol = sheet.range(keycell).end('right').column
    bondname = sheet.range('B4').value[13:-1]
    tempdf = sheet.range(keycell,(maxrow,maxcol)).options(pd.DataFrame, index=False).value
    tempdf['종목명'] = bondname
    tempdf.set_index(['일자','종목명'], inplace=True)
    tempdf_notnull = tempdf[tempdf['민평3사 수익률(산출일) 당일'].notnull()]
    df = df.append(tempdf_notnull)
df


In [ ]:
# 생성한 시트들을 loop돌면서 시트명(KR코드)을 받아와서 각시트 조회 종목코드셀('B3')에 넣어주기
# 이렇게 2단계로 진행해야 엑셀측에서 이상한 에러가 안난다.
sheets = xw.Book(r'db_bonddata.xlsx').sheets
for sheet in sheets:
    sheet.range('B3').value = sheet.name
# 전종목 데이터 조회 엑셀파일 완성

In [ ]:
# 생성한 시트들을 loop돌면서 시트명(KR코드)을 받아와서 각시트 조회 종목코드셀('B3')에 넣어주기
# 이렇게 2단계로 진행해야 엑셀측에서 이상한 에러가 안난다.
sheets = xw.Book(r'db_bonddata.xlsx').sheets
for sheet in sheets:
    sheet.range('B3').value = sheet.name
# 전종목 데이터 조회 엑셀파일 완성

In [ ]:
sheet = xw.Book(r'db_bonddata.xlsx').sheets[-1]
keycell = 'A9'
maxrow = sheet.range(keycell).end('down').row
maxcol = sheet.range(keycell).end('right').column
bondname = sheet.range('B4').value[13:-1]
df = sheet.range(keycell,(maxrow,maxcol)).options(pd.DataFrame, index=False).value
df['종목명'] = bondname
df.set_index(['일자','종목명'], inplace=True)
df

In [ ]:
max_row = sht_bond_data.range('B9').end('down').row
bond_name = sht_bond_data.range('B4').value[13:-1]
df_bond_data = sht_bond_data.range((9,1),(max_row,34)).options(pd.DataFrame, index=False).value
df_bond_data['종목명'] = bond_name
df_bond_data.set_index(['일자','종목명'], inplace=True)
df_bond_data

In [ ]:
df_new = df[df['민평3사 수익률(산출일) 당일'].notnull()]
df_new

In [ ]:
import time
sheet = xw.Book(r'db_bonddata.xlsx').sheets[0]
copied = sheet.range('A1','AI9').copy()

for code, name in zip(codes, names):
    # newsheet = originalsheet.copy(name=name)
    newsheet = sheets.add(name=name)
    time.sleep(0.5)
    newsheet.range('A1').paste(copied)
    time.sleep(0.5)
    newsheet.range('B3').value = code
    time.sleep(0.5)
    # time.sleep(1)
    # newsheet.range('B3').value = code
    # time.sleep(1)

In [ ]:
sheets_ah = xw.Book(r'excel_db.xlsx').sheets

In [ ]:
sht_auction_calendar = sheets_ah[0]
sht_rates = sheets_ah[1]
sht_bond_data = sheets_ah[2]
sht_bond_balance = sheets_ah[3]

In [ ]:
max_row = sht_auction_calendar.range('A2').end('down').row
df_auction_calendar = sht_auction_calendar.range((2,1),(max_row,16)).options(pd.DataFrame, index=False).value

In [ ]:
max_row = sht_rates.range('A4').end('down').row
df_rates = sht_rates.range((4,1),(max_row,11)).options(pd.DataFrame, index=False).value
df_rates

In [ ]:
max_row = sht_bond_balance.range('B9').end('down').row
bond_name = sht_bond_data.range('B4').value[13:-1]
df_bond_balance = sht_bond_balance.range((9,1),(max_row,2)).options(pd.DataFrame, index=False).value
df_bond_balance['종목명'] = bond_name
df_bond_balance.set_index(['일자','종목명'], inplace=True)
df_bond_balance

In [ ]:
df_bond_balance.info()

In [ ]:
df = pd.DataFrame()
for sht in tqdm(sheets_ktb):
    krcode = sht.range('F1').value
    bond_name = sht.range('G1').value
    maturity = sht.range('I1').value
    max_row = sht.range('B3').end('down').row
    temp_df = sht.range((3,1),(max_row,40)).options(pd.DataFrame, index=False).value
    temp_df['종목코드'] = krcode
    temp_df['종목명'] = bond_name[13:-1]
#     sht.name = bond_name[13:-1]
    temp_df['잔존만기'] = (maturity - temp_df['일자']).dt.days
    temp_df.set_index(['일자','종목명'], inplace=True)
    df = df.append(temp_df)


In [ ]:
df

In [ ]:
df.columns = [
    '민평금리',
    '외국인매수','은행매수','보험기금매수','공모매수','사모매수','종금매수','정부매수','기타법인매수','개인매수',
    '외국인매도','은행매도','보험기금매도','공모매도','사모매도','종금매도','정부매도','기타법인매도','개인매도',
    '상장잔액증감', '장외거래량',
    '대차전일잔량','대차거래','대차상환','대차잔량',
    '증권대여','보험대여','투신대여','은행대여','연기금대여','외국인대여','기타대여',
    '증권차입','보험차입','투신차입','은행차입','연기금차입','외국인차입','기타차입',
    '종목코드','잔존만기'
]

In [ ]:
df.fillna(0, inplace=True)

In [ ]:
df['외국인순매수'] = df['외국인매수'] - df['외국인매도']
df['은행순매수'] = df['은행매수'] - df['은행매도']
df['보험기금순매수'] = df['보험기금매수'] - df['보험기금매도']
df['공모순매수'] = df['공모매수'] - df['공모매도']
df['사모순매수'] = df['사모매수'] - df['사모매도']
df['종금순매수'] = df['종금매수'] - df['종금매도']
df['정부순매수'] = df['정부매수'] - df['정부매도']
df['기타법인순매수'] = df['기타법인매수'] - df['기타법인매도']
df['개인순매수'] = df['개인매수'] - df['개인매도']

In [ ]:
df.columns

In [ ]:
df[['외국인매수', '은행매수', '보험기금매수', '공모매수', '사모매수', '종금매수', '정부매수',
       '기타법인매수', '개인매수', '외국인매도', '은행매도', '보험기금매도', '공모매도', '사모매도', '종금매도',
       '정부매도', '기타법인매도', '개인매도', '상장잔액증감', '장외거래량', '대차전일잔량', '대차거래', '대차상환', '대차잔량', '증권대여',
       '보험대여', '투신대여', '은행대여', '연기금대여', '외국인대여', '기타대여', '증권차입', '보험차입',
       '투신차입', '은행차입', '연기금차입', '외국인차입', '기타차입', '잔존만기', '외국인순매수',
       '은행순매수', '보험기금순매수', '공모순매수', '사모순매수', '종금순매수', '정부순매수', '기타법인순매수',
       '개인순매수']] = df[['외국인매수', '은행매수', '보험기금매수', '공모매수', '사모매수', '종금매수', '정부매수',
       '기타법인매수', '개인매수', '외국인매도', '은행매도', '보험기금매도', '공모매도', '사모매도', '종금매도',
       '정부매도', '기타법인매도', '개인매도',  '상장잔액증감','장외거래량', '대차전일잔량', '대차거래', '대차상환', '대차잔량', '증권대여',
       '보험대여', '투신대여', '은행대여', '연기금대여', '외국인대여', '기타대여', '증권차입', '보험차입',
       '투신차입', '은행차입', '연기금차입', '외국인차입', '기타차입', '잔존만기', '외국인순매수',
       '은행순매수', '보험기금순매수', '공모순매수', '사모순매수', '종금순매수', '정부순매수', '기타법인순매수',
       '개인순매수']].astype(np.int64)

In [ ]:
df

In [ ]:
df.sort_index(inplace=True)

In [ ]:
df.loc(axis=0)['2021-1-2':'2021-4-19','20-6'][['외국인순매수', '은행순매수', '보험기금순매수', '공모순매수', '사모순매수', '종금순매수', '정부순매수', '기타법인순매수', '개인순매수']].cumsum().plot()

In [ ]:
df.to_pickle('ktb.pkl')

In [ ]:
import sqlite3
con = sqlite3.connect(r'D:\OneDrive\07_Development\infomax_xlwings_practice/ktb.db')
df.to_sql('ktb', con, if_exists='replace')


In [ ]:
con.close()

In [ ]:
df.to_csv("df.csv")

In [ ]:
df['공모매수'].value_counts()

In [ ]:
df_new = df.astype(int)

In [ ]:
df2 = df.unstack()

In [ ]:
df2

In [ ]:
df2.to_csv("df2.csv")

In [ ]:
df2['2021-01-02':]['외국인순매수','19-8'].plot()

In [ ]:
df3.columns()

In [ ]:
df.isna().sum()

In [ ]:
df.fillna(0, inplace=True)

In [ ]:
df

In [ ]:
df.sort_index(inplace=True)

In [ ]:
df.loc(axis=0)['2021-1-2':'2021-4-19',:]['외국인순매수'].plot()

In [ ]:
sns.heatmap(df.loc(axis=0)['2021-4-1':'2021-4-19',:]['외국인순매수'].unstack())

In [ ]:
import plotly.graph_objects as go
fig = go.Figure(data=go.Bar(y=[2, 3, 1]))
fig.show()

In [ ]:
import plotly.graph_objects as go
fig = go.FigureWidget(data=go.Bar(y=[2, 3, 1]))
fig

In [ ]:
import plotly.express as px
df = px.data.stocks()
df

In [ ]:
import sqlite3

In [ ]:
con = sqlite3.connect(r'C:\Users\infomax\Documents\DT\infomax_xlwings_practice/ktb2.db')

In [ ]:
df2.to_sql('ktb2', con)

In [ ]:
reset_df['민평금리'].value_counts()

In [ ]:
reset_df[reset_df['민평금리'].isna()]['종목명'].value_counts()

In [ ]:
df.

In [ ]:
df[df['민평금리'].isna()].index

In [ ]:
maturity = sht.range('I1').value
maturity

In [ ]:
df = df.reorder_levels([1,0], axis=0)

In [ ]:
df.loc[(slice(pd.Timestamp('2021-01-04'),pd.Timestamp('2021-04-19')))]

In [ ]:
df.info()

In [ ]:
df

In [ ]:
df.shape

In [ ]:
df['민평3사 수익률(산출일) 당일'].plot()

In [ ]:
import pickle
df.to_pickle('ktb_pickle_210416')

In [ ]:
result_df[]

In [ ]:
result_df

In [ ]:
krcode = sht.range('F1').value
bond_name = sht.range('A2').value
max_row = sht.range('B4').end('down').row
df = sht.range((3,1),(max_row,38)).options(pd.DataFrame, index=False).value
df['종목코드'] = krcode
df['종목명'] = bond_name
df.set_index(['일자','종목코드'], inplace=True)


In [ ]:
bond_name = sht.range('A2').value

In [ ]:
max_row = sht.range('B4').end('down').row

In [ ]:
buy = ['b'+ str(j) for j in range(1,10)]
sell = ['s'+ str(j) for j in range(1,10)]

In [ ]:
df = sht.range((3,1),(max_row,38)).options(pd.DataFrame, index=False).value

In [ ]:
df['종목코드'] = krcode
df['종목명'] = bond_name
df

In [ ]:
df.set_index(['일자','종목코드'], inplace=True)
df

In [ ]:
df.plot()

In [ ]:
for sheet in sheets_ktb:
    print(sheet.name)

In [ ]:
sheet_flows = xw.Book(r'infomax_data.xlsm').sheets['flows_kr']
df_flows_kr = pd.DataFrame(
    sheet_flows.range('D14').expand().value,
    index=pd.DatetimeIndex(sheet_flows.range('C14').expand('down').value, name="Date"),
    columns=sheet_flows.range('D13').expand('right').value
    )
df_sorted_flows_kr = df_flows_kr.sort_index()
df_sorted_flows_kr

In [ ]:
sheet_rates = xw.Book(r'infomax_data.xlsm').sheets['rates_kr']

In [ ]:
df_rates_kr = pd.DataFrame(
    sheet_rates.range('D14').expand().value,
    index=pd.DatetimeIndex(sheet_rates.range('C14').expand('down').value, name="Date"),
    columns=sheet_rates.range('D13').expand('right').value
    )
df_sorted_rates_kr = df_rates_kr.sort_index()
df_sorted_rates_kr

In [ ]:
df_sorted_rates_kr.to_csv('rates_kr.csv')

In [ ]:
df_rates_kr_diff = df_sorted_rates_kr.diff().dropna()

In [ ]:
df_rates_kr_diff['KTB10Y'].plot.hist(bins=20)

In [ ]:
df_m = df_rates_kr_diff.copy()
df_m['month'] = df_m.index.month

In [ ]:
df_m['year'] = df_m.index.year


In [ ]:
df_m['weekday'] = df_m.index.weekday

In [ ]:
fig, ax = plt.subplots(figsize=(15,6))
# sns.heatmap(df_m.groupby(['month','year']).mean()['KTB10Y'].unstack(level=0))
# sns.heatmap(df_m[df_m['year']==2020].groupby(['month','weekday']).mean()['KTB10Y'].unstack(level=0), center=0, cmap="bwr")
sns.heatmap(df_m.groupby(['year','weekday']).mean()['KTB10Y'].unstack(level=0), center=0, cmap="bwr")
# sns.heatmap(df_m.groupby(['weekday','year','month']).mean()['KTB10Y'].unstack(level=0), center=0, cmap="bwr")

In [ ]:
df_year_month = df_m['KTB10Y']

In [ ]:
ax = sns.heatmap(df_rates_kr_diff['KTB10Y'],

In [ ]:
df_rates_kr_diff['Y']

In [ ]:
df_rates_kr.info()

In [ ]:
rates = xw.Book(r'infomax_sample.xlsx').sheets['국내금리']
#rates = xw.Book(r'infomax_sample.xlsx').sheets[0]
deposit = xw.Book(r'infomax_sample.xlsx').sheets['금융기관수신고']

In [ ]:
#df = pd.DataFrame(xw.Range('a4:i4969').value,columns=['DATE','KTB1Y','KTB2Y','KTB3Y','KTB5Y','KTB10Y','KTB20Y','KTB30Y','KTB50Y'])
#df = pd.DataFrame(xw.Range('a4').expand().value,columns=['DATE','KTB1Y','KTB2Y','KTB3Y','KTB5Y','KTB10Y','KTB20Y','KTB30Y','KTB50Y'])
df_rates = pd.DataFrame(
    rates.range('E10').expand().value, 
    index=pd.DatetimeIndex(rates.range('D10').expand('down').value), 
    columns=pd.MultiIndex.from_arrays(rates.range('E8:E9').expand('right').value, names=['name','maturity'])
)
deposit_columns=deposit.range('A3').expand('right').value
deposit_columns.pop(1)
df_deposit = pd.DataFrame(
    data=deposit.range('B5').expand().value,
    index=pd.DatetimeIndex(deposit.range('A5').expand('down').value),
    columns=deposit_columns
)

In [ ]:
df_rates.replace({0:np.nan}, inplace=True)

In [ ]:
df_rates['국고'].plot()

In [ ]:
df_deposit.plot()

In [ ]:
plt.rcParams["font.family"] = 'NanumGothic'

In [ ]:
df

In [ ]:
df.index

In [ ]:
df

In [ ]:
plt.figure(figsize=(12,8))
df['국고'].plot()
#plt.plot(df.index, df['공사AAA'])
plt.legend(loc='best')
plt.grid()
plt.show()

In [ ]:
df.loc['2000-01-15':'2000-12-31'].plot()

In [ ]:
plt.plot(df['국고'])
plt.title('가격의 변화')

In [ ]:
print (plt.rcParams['font.serif']) 
print (plt.rcParams['font.sans-serif']) 
print (plt.rcParams['font.monospace']) 